In [ ]:
using Gmsh: gmsh
using GridapGmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using Plots

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0,0.0,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym - I4_vol

In [ ]:
const L = 150
const B_b = 31.7
const B_s = 3.17
const CrckPos = 30
const ch = 15
const cw = 0.1
const lsp = 3*cw
const hfc = lsp/4
const hf = lsp/4
const h = B_b/15
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h, 1)
gmsh.model.geo.addPoint(CrckPos, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(CrckPos, ch, 0.0, h, 3)
gmsh.model.geo.addPoint(CrckPos+cw, ch, 0.0, h, 4)
gmsh.model.geo.addPoint(CrckPos+cw, 0, 0.0, h, 5)
gmsh.model.geo.addPoint(L, 0.0, 0.0, h, 6)
gmsh.model.geo.addPoint(L, B_b, 0.0, h, 7)
gmsh.model.geo.addPoint(L, B_b + B_s, 0.0, h, 8)
gmsh.model.geo.addPoint(0, B_b + B_s, 0.0, h, 9)
gmsh.model.geo.addPoint(0, B_b, 0.0, h, 10)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 9, 8)
gmsh.model.geo.addLine(9, 10, 9)
gmsh.model.geo.addLine(10, 1, 10)
gmsh.model.geo.addLine(7, 10, 11)

gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,11,10],1) 
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.addCurveLoop([-11,7,8,9],2) 
gmsh.model.geo.addPlaneSurface([2], 2)

gmsh.model.addPhysicalGroup(2, [1],1)
gmsh.model.addPhysicalGroup(2, [2],2)
gmsh.model.addPhysicalGroup(1, [1,2,3,4,5,6,7,8,9,10],1)
gmsh.model.setPhysicalName(2, 1, "BGlass")
gmsh.model.setPhysicalName(2, 2, "SSteel")
gmsh.model.setPhysicalName(1, 1, "Boundary")

gmsh.model.mesh.field.add("Box", 20)
gmsh.model.mesh.field.setNumber(20, "VIn", hf)
gmsh.model.mesh.field.setNumber(20, "VOut", h)
gmsh.model.mesh.field.setNumber(20, "XMin", 0.8*CrckPos)
gmsh.model.mesh.field.setNumber(20, "XMax",2.3*CrckPos)
gmsh.model.mesh.field.setNumber(20, "YMin",0.85*ch)
gmsh.model.mesh.field.setNumber(20, "YMax",B_b)
gmsh.model.mesh.field.setAsBackgroundMesh(20)


gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("BiMaterial.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("BiMaterial.msh")
writevtk(model,"BiMaterial")

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension)
const BGlass_tag = get_tag_from_name(labels,"BGlass")
const SSteel_tag = get_tag_from_name(labels,"SSteel")

In [ ]:
const E_BGlass = 64e3
const ν_BGlass = 0.2
G₁₂_BGlass = E_BGlass/(2*(1+ν_BGlass))

const E_SSteel = 193e3
const ν_SSteel = 0.29
G₁₂_SSteel = E_SSteel/(2*(1+ν_SSteel))

#### Input fracture parameters

In [ ]:
const Gc_BGlass = 0.4
const Gc_SSteel = 50
const η = 1e-8

In [ ]:
function Gc(s_id,tag)
        if tag == BGlass_tag          
            return  Gc_BGlass*s_id
        elseif tag == SSteel_tag    
            return Gc_SSteel*s_id
        end
end

#### Input Thermal parameter 

In [ ]:
const α_BGlass = 3.25e-6
const α_SSteel = 17.3e-6
const T0 = 300.0

## Constitutive Matrices

### Elastic stiffness tensor

In [ ]:
function ElasFourthOrderConstTensor(E ,ν , PlanarState)
# 1 for Plane Stress and 2 Plane Strain Condition
if PlanarState == 1
C1111 = E /(1 -ν *ν )
C1122 = (ν *E ) /(1 -ν *ν )
C1112 = 0.0
C2222 = E /(1 -ν *ν )
C2212 = 0.0
C1212 = E /(2*(1+ν ) )
elseif PlanarState == 2
C1111 = (E *(1 -ν *ν ) ) /((1+ν ) *(1 -ν -2*ν *ν ) )
C1122 = (ν *E ) /(1 -ν -2*ν *ν )
C1112 = 0.0
C2222 = (E *(1 -ν ) ) /(1 -ν -2*ν *ν )
C2212 = 0.0
C1212 = E /(2*(1+ν ) )
end
C_ten = SymFourthOrderTensorValue(C1111 , C1112 , C1122 , C1112 ,
C1212 , C2212 , C1122 , C2212 , C2222)
return C_ten
end

In [ ]:
const C_BGlass = ElasFourthOrderConstTensor(E_BGlass ,ν_BGlass ,1)
const C_SSteel = ElasFourthOrderConstTensor(E_SSteel ,ν_BGlass ,1)

In [ ]:
function σ_elas(ε, T, tag)  
 if tag == BGlass_tag
      return C_BGlass ⊙ (ε - (α_BGlass*I2)*(T-T0))
  elseif tag == SSteel_tag
      return C_SSteel ⊙ (ε -  (α_SSteel*I2)*(T-T0))
  end
end

In [ ]:
function σ_Temp(T,tag)
 if tag == BGlass_tag
      return C_BGlass ⊙ ((α_BGlass*I2)*(T-T0))
  elseif tag == SSteel_tag
      return C_SSteel ⊙ ((α_SSteel*I2)*(T-T0))
  end
end

In [ ]:
function σ_TempMod(ε_in,s_in,T_in,tag)
    if tag == BGlass_tag
        ch = ε_in-α_BGlass*(T_in-T0)*I2
        if tr(ch) >= 0
            return (s_in^2 + η)*σ_Temp(T_in,tag) 
        elseif tr(ch) < 0
            return (s_in^2 + η) *I4_dev ⊙ σ_Temp(T_in,tag)   + I4_vol⊙ σ_Temp(T_in,tag) 
        end
  elseif tag == SSteel_tag
        ch = ε_in-α_SSteel*(T_in-T0)*I2
        if tr(ch) >= 0
            return (s_in^2 + η)*σ_Temp(T_in,tag) 
        elseif tr(ch) < 0
            return (s_in^2 + η) *I4_dev ⊙ σ_Temp(T_in,tag)   + I4_vol⊙ σ_Temp(T_in,tag) 
        end
    end
end

In [ ]:
function σ_Tot(ε,tag)  
 if tag == BGlass_tag
      return C_BGlass ⊙ ε 
  elseif tag == SSteel_tag
      return C_SSteel ⊙ ε 
  end
end

In [ ]:
function σ_elasMod(ε, ε_in, s_in,T_in,tag)
    if tag == BGlass_tag
        ch = ε_in-α_BGlass*(T_in-T0)*I2
        if tr(ch) >= 0
            return ((s_in^2 + η)*σ_elas(ε,T_in,tag))
        elseif tr(ch) < 0
            return (((s_in^2 + η) *I4_dev ⊙ σ_elas(ε,T_in,tag)) + I4_vol⊙ σ_elas(ε,T_in,tag))
        end
  elseif tag == SSteel_tag
        ch = ε_in-α_SSteel*(T_in-T0)*I2
        if tr(ch) >= 0
            return ((s_in^2 + η)*σ_elas(ε,T_in,tag))
        elseif tr(ch) < 0
            return (((s_in^2 + η) *I4_dev ⊙ σ_elas(ε,T_in,tag)) + I4_vol⊙ σ_elas(ε,T_in,tag))
        end
    end
end

In [ ]:
function σ_TotMod(ε, ε_in, s_in,T_in,tag)
    if tag == BGlass_tag
        if tr(ε_in-α_BGlass*(T_in-T0)*I2) >= 0
            return (s_in^2 + η)*σ_Tot(ε,tag) 
        elseif tr(ε_in-α_BGlass*(T_in-T0)*I2) < 0
            return (s_in^2 + η) *I4_dev ⊙ σ_Tot(ε,tag)  + I4_vol⊙ σ_Tot(ε,tag) 
        end
  elseif tag == SSteel_tag
        if tr(ε_in-α_SSteel*(T_in-T0)*I2) >= 0
            return (s_in^2 + η)*σ_Tot(ε,tag) 
        elseif tr(ε_in-α_SSteel*(T_in-T0)*I2) < 0
            return (s_in^2 + η) *I4_dev ⊙ σ_Tot(ε,tag)  + I4_vol⊙ σ_Tot(ε,tag)
        end
    end
end

In [ ]:
function ψPos(ε_in,T_in,tag)
    if tag == BGlass_tag
        ch =  ε_in-α_BGlass*(T_in-T0)*I2
        if tr(ch) >= 0
            return 0.5*(ch ⊙ σ_elas(ε_in,T_in,tag))
        elseif tr(ch) < 0
            return 0.5*((I4_dev ⊙ σ_elas(ε_in,T_in,tag)) ⊙ (I4_dev ⊙ ch))
        end
  elseif tag == SSteel_tag
        ch = ε_in-α_SSteel*(T_in-T0)*I2
        if tr(ch) >= 0
            return 0.5*(ch ⊙ σ_elas(ε_in,T_in,tag))
        elseif tr(ch) < 0
            return 0.5*((I4_dev ⊙ σ_elas(ε_in,T_in,tag)) ⊙ (I4_dev ⊙ ch))
        end
    end   
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
order = 1
degree = 2*order

In [ ]:
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
reffe_PF = ReferenceFE(lagrangian,Float64,order)
V0_PF = TestFESpace(model,reffe_PF;
  conformity=:H1)
U_PF = TrialFESpace(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["Boundary"],
          dirichlet_masks=[(false,false)])
uh = zero(V0_Disp)

In [ ]:
function  stepPhaseField(uh_in,ψPlusPrev_in)
        
        a_PF(s,ϕ) = ∫( Gc∘(shId,mat_tags)*lsp*∇(ϕ)⋅ ∇(s) + 2*ψPlusPrev_in*s*ϕ  + (Gc∘(shId,mat_tags)/lsp)*s*ϕ )*dΩ
        b_PF(ϕ) = ∫( (Gc∘(shId,mat_tags)/lsp)*ϕ )*dΩ
        op_PF = AffineFEOperator(a_PF,b_PF,U_PF,V0_PF)
        sh_out = solve(op_PF)           
    
    return sh_out
    
end

In [ ]:
 function   stepDisp(uh_in,sh_in,T_in,tag)
        uApp1(x) = VectorValue(0.0,0.0)
        U_Disp = TrialFESpace(V0_Disp,[uApp1])
        a(u,v) = ∫( (ε(v) ⊙ (σ_TotMod∘(ε(u),ε(uh_in),sh_in,T_in,tag))))*dΩ
        b(v) = ∫( (ε(v) ⊙ (σ_TempMod∘(ε(uh_in),sh_in,T_in,tag))))*dΩ
        op = AffineFEOperator(a,b,U_Disp,V0_Disp)         
        uh = solve(op)                
        uh_out =  uh
    
    return uh_out
end

In [ ]:
PseudoVelU = 0.0
delt = 0.5
t = 0
tMax = 500.0
delT = delt
TAppMax = 0.0
innerMax = 10
count = 0
tol = 1e-8

Load = Float64[]
Displacement = Float64[]

push!(Load, 0.0)
push!(Displacement, 0.0)

ψPlusPrev = CellState(0.0,dΩ) 
sPrev = CellState(1.0,dΩ)
sh = project(sPrev,model,dΩ,order)
while t .<= tMax
    ThPrev = CellState(T0 -count*delT,dΩ)
    Th = project(ThPrev,model,dΩ,order)
    
    count = count .+ 1
    t = t + delt
    vApp = PseudoVelU*t
    TApp = T0 - count*delT
    
   
    
    print("\n Entering time step$count :", float(t))
    
   for inner = 1:innerMax   
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
     RelErr = abs(sum(∫( Gc∘(shId,mat_tags)*lsp*∇(sh)⋅ ∇(sh) + 2*ψhPlusPrev*sh*sh  + (Gc∘(shId,mat_tags)/lsp)*sh*sh)*dΩ - ∫( (Gc∘(shId,mat_tags)/lsp)*sh)*dΩ))/abs(sum(∫( (Gc∘(shId,mat_tags)/lsp)*sh)*dΩ))
        print("\n Relative error = ",float(RelErr))
        sh = stepPhaseField(uh,ψhPlusPrev) 
        uh = stepDisp(uh,sh,Th,mat_tags)
            ψhPos_in = ψPos∘(ε(uh),Th,mat_tags)      
        
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
  
       if RelErr < tol
          break 
       end      
    end
     
    if mod(count,10) == 0
         writevtk(Ω,"results_PhaseFieldThermoElastic$count",cellfields=
        ["s"=>sh])
    end
    end

In [ ]:
     writevtk(Ω,"results_PhaseFieldThermoElastic$count",cellfields=
        ["s"=>sh])